# LangGraph 에 자주 등장하는 Python 문법


## 목차 
- TypedDict
- Annotated 
- add_messages

## TypedDict

`dict`와 `TypedDict`의 비교와 사용 이유

1. `dict`와 `TypedDict`의 주요 차이점:

   a) 타입 검사:
      - `dict`: 런타임에 타입 검사 x 
      - `TypedDict`: 정적 타입 검사를 제공. 즉, 코드 작성 시 IDE나 타입 체커가 오류를 미리 catch 가능 

   b) 키와 값의 타입:
      - `dict`: 키와 값의 타입을 일반적으로 지정 (예: Dict[str, str]).
      - `TypedDict`: 각 키에 대해 구체적인 타입을 지정

   c) 유연성:
      - `dict`: 런타임에 키를 추가하거나 제거
      - `TypedDict`: 정의된 구조를 따라야 함. 추가적인 키는 타입 오류를 발생

2. `TypedDict`가 `dict` 대신 사용되는 이유:

   a) 타입 안정성: 
      - `TypedDict` : 엄격한 타입 검사 -> 잠재적인 버그 미리 방지

   c) IDE 지원:
      - `TypedDict` : IDE에서 자동 완성 및 타입 힌트를 더 정확하게 제공받을 수 있음

   d) 문서화:
      - `TypedDict`는 코드 자체가 문서의 역할을 하여 딕셔너리의 구조를 명확히 보여줌

In [8]:
# TypedDict와 Dict의 차이점 예시
from typing import Dict, TypedDict

# 일반적인 파이썬 딕셔너리(dict) 사용
sample_dict: Dict[str, str] = {
    "name": "철수",
    "age": "10",  # 문자열로 저장 
    "job": "학생",
}


# TypedDict 사용
class Person(TypedDict):
    name: str
    age: int  # 정수형으로 명시
    job: str


typed_dict: Person = {"name": "영희", "age": 10, "job": "학생"}

In [10]:
sample_dict

{'name': '철수', 'age': '10', 'job': '학생'}

In [9]:
typed_dict

{'name': '영희', 'age': 10, 'job': '학생'}

- VS Code 등의 IDE에서 타입 검사 기능을 활성화하면, 
- 타입 불일치와 정의되지 않은 키 추가를 오류로 표시합니다.
- 정적 타입 검사기를 사용하면 다음과 같은 오류 메시지를 볼 수 있습니다.

# Annotated

- 타입 힌트에 메타데이터를 추가할 수 있다 
- 다룰 변수에 라벨을 달아준다고 생각

### `Annotated`를 사용하는 주요 이유

**추가 정보 제공(타입 힌트) / 문서화** 

- 타입 힌트에 추가적인 정보를 포함시킬 수 있음 
    + 이는 코드를 읽는 사람이나 도구에 더 많은 컨텍스트를 제공
- 코드에 대한 추가 설명을 타입 힌트에 직접 포함시킬 수 있음

`name: Annotated[str, "이름"]`

`age: Annotated[int, "나이"]`





### Annotated 주요 기능(사용 이유)

1. **추가 정보 제공**: 타입 힌트에 메타데이터를 추가 -> 더 상세한 정보 제공
2. **문서화**: 코드 자체에 추가 설명을 포함시켜 -> 문서화 효과
3. **유효성 검사**: 특정 라이브러리(예: Pydantic)와 함께 사용하여 -> 데이터 유효성 검사 수행 가능 
4. **프레임워크 지원**: 일부 프레임워크(예: LangGraph)에서는 `Annotated`를 사용하여 특정 동작을 정의 함 : 기능 구현을 위해 써야함

**기본 문법**

- `Type`: 기본 타입 (예: `int`, `str`, `List[str]` 등)
- `metadata1`, `metadata2`, ...: 추가하고자 하는 메타데이터
  
예시 코드

```python
from typing import Annotated

variable: Annotated[Type, metadata1, metadata2, ...]
```

### 사용 예시 

In [14]:
from typing import Annotated

name: Annotated[str, "사용자 이름"]
age: Annotated[int, "사용자 나이 (0-150)"]

In [ ]:
from typing import Annotated, List
from pydantic import Field, BaseModel, ValidationError


class Employee(BaseModel):
    id: Annotated[int, Field(..., description="직원 ID")]                                   # ... 의 의미: 반드시 누락 되면 안된다. 해당 변수가 
    name: Annotated[str, Field(..., min_length=3, max_length=50, description="이름")]
    age: Annotated[int, Field(gt=18, lt=65, description="나이 (19-64세)")]
    salary: Annotated[float, Field(gt=0, lt=10000, description="연봉 (단위: 만원, 최대 10억)")]
    skills: Annotated[List[str], Field(min_items=1, max_items=10, description="보유 기술 (1-10개)")]


In [19]:
# 1. 유효한 데이터로 인스턴스 생성
try:
    valid_employee = Employee(
        id=1, 
        name="김현우", 
        age=30, 
        salary=1000, 
        skills=["Python", "LangChain"]
    )
    print("유효한 직원 데이터:", valid_employee)
    
except ValidationError as e:
    print("유효성 검사 오류:", e)


유효한 직원 데이터: id=1 name='김현우' age=30 salary=1000.0 skills=['Python', 'LangChain']


In [21]:
# 2. 유효하지 않은 데이터로 인스턴스 생성 시도
try:
    invalid_employee = Employee(                
        # id 없으니 오류         
        name="현",  # 이름이 너무 짧음                    
        age=17,  # 나이가 범위를 벗어남
        salary=20000,  # 급여가 범위를 벗어남
        skills="Python",  # 리스트가 아님
    )
except ValidationError as e:
    print("유효성 검사 오류:")
    for error in e.errors():
        print(f"- {error['loc'][0]}: {error['msg']}")

유효성 검사 오류:
- id: Field required
- name: String should have at least 3 characters
- age: Input should be greater than 18
- salary: Input should be less than 10000
- skills: Input should be a valid list


# add_messages

### LangGraph에서의 사용(add_messages)

`add_messages` : LangGraph 에서 메시지를 리스트에 추가하는 함수

In [22]:
from typing import Annotated, TypedDict
from langgraph.graph import add_messages


class MyData(TypedDict):
    messages: Annotated[list, add_messages]

**참고**

1. `Annotated`는 Python 3.9 이상에서 사용 가능
2. 런타임에는 `Annotated`가 무시되므로, 실제 동작에는 영향을 주지 않음
3. 타입 검사 도구나 IDE가 `Annotated`를 지원해야 그 효과를 볼 수 있음

In [24]:
from langchain_core.messages import AIMessage, HumanMessage
from langgraph.graph import add_messages

# 기본 사용 예시
msgs1 = [HumanMessage(content="안녕하세요?", id="1")]
msgs2 = [AIMessage(content="반갑습니다~", id="2")]

result1 = add_messages(msgs1, msgs2)

In [26]:
result1

[HumanMessage(content='안녕하세요?', additional_kwargs={}, response_metadata={}, id='1'),
 AIMessage(content='반갑습니다~', additional_kwargs={}, response_metadata={}, id='2')]


주석이 없는 상태 키는 각 업데이트에 의해 덮어쓰여져 가장 최근의 값이 저장 됨

`add_messages` 함수
   - 2개의 인자(left, right)를 받음 
   - 좌, 우 메시지를 병합하는 방식으로 동작

**주요 기능**
   - 두 개의 메시지 리스트를 병합
   - 기본적으로 "append-only" 상태를 유지
   - 동일한 ID를 가진 메시지가 있을 경우, 새 메시지로 기존 메시지를 대체

**동작 방식**
   - `right`의 메시지 중 `left`에 동일한 ID를 가진 메시지가 있으면, `right`의 메시지로 대체
   - 그 외의 경우 `right`의 메시지가 `left`에 추가됩니다.

**매개변수**
   - `left` (Messages): 기본 메시지 리스트
   - `right` (Messages): 병합할 메시지 리스트 또는 단일 메시지

**반환값**
   - `Messages`: `right`의 메시지들이 `left`에 병합된 새로운 메시지 리스트

In [28]:
# 동일한 ID를 가진 메시지에 대해 기존 메시지는 제거되고 현재 메시지로 대체
msgs1 = [HumanMessage(content="안녕하세요?", id="1")]
msgs2 = [HumanMessage(content="반갑습니다~", id="1")]

result2 = add_messages(msgs1, msgs2)

In [29]:
result2

[HumanMessage(content='반갑습니다~', additional_kwargs={}, response_metadata={}, id='1')]